# Calculating differences between accessibility scenarios

This notebook outlines a process to loop through a collection of accessibility rasters for each investment scenario and quantify the difference in accessibility associated with each relative to a comparison set of accessibility rasters.

## Imports

In [4]:
import os, sys, glob
import re
from datetime import date

from datetime import datetime

import pprint
from pprint import pprint
from itertools import islice

import common_rasterio_ops as rast_ops

import rasterio
from rasterio import features, transform
from rasterio.mask import mask
from rasterio.transform import Affine
from rasterio.io import MemoryFile

import numpy as np
from numpy import ma
import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape, box, Polygon

sys.path.append('../../src/')
from gostrocks.src.GOSTRocks.misc import tPrint

## Setup

File paths

In [5]:
geo_pth = r'P:\PAK\GEO'
data_dir = r'../../data'

rds_dir = r'roads'
acc_dir = r'access'
tab_dir = r'tabular'

dif_dir = r'access//difs'
ph_dir = r'access//personhours'

Dates

In [6]:
today = date.today().strftime("%d%m%y")

In [7]:
data_date = '211028'

Projections

In [8]:
dest_crs = 'EPSG:32642'

Resolution of rasters

In [9]:
res = '31m'

## Load in files

In [18]:
# populate lists

proposed_road_list = sorted([os.path.join(data_dir,acc_dir, f'upgrade//{res}', f) 
               for f in os.listdir(os.path.join(data_dir,acc_dir,f'upgrade//{res}')) \
#                if 'dry' in f]) # example filtering code
               if f.endswith(".tif")])

status_quo_road_list = sorted([os.path.join(data_dir,acc_dir, f'current//master', f) 
               for f in os.listdir(os.path.join(data_dir,acc_dir,f'current//master')) 
#                if 'dry' in f and 'walk' not in f]) # example filtering code
               if f.endswith(".tif") and 'walk' not in f])

# remove any extraneous files
status_quo_road_list.remove('../../data\\access\\current//master\\DEM_KPK_UTM.tif')

In [19]:
len(proposed_road_list)

5688

In [20]:
# # Remove roads already evaluated, if doing this in stages. If doing this all at once, ignore
# # make a list of roads IDs for roads already evaluated/processed, captured in an outside CSV

# processed_rds = pd.read_csv(os.path.join(data_dir,tab_dir,f'final//roads_evaluation_data_combined_211208.csv'))
# exclusion_lst = list(processed_rds.Road_ID.astype(str))

# # filter down proposed_roads_list using exclusion_lst
# proposed_road_list = [file for file in proposed_road_list \
#                         if re.search('^[0-9]+',os.path.basename(file))[0] not in exclusion_lst]

# print(len(exclusion_lst))

In [21]:
len(proposed_road_list)

5688

In [22]:
proposed_road_list[::150]

['../../data\\access\\upgrade//31m\\10_LowerChitral_upgrade_dry_District_HQs_access.tif',
 '../../data\\access\\upgrade//31m\\12_Kohistan_upgrade_dry_education_boys_middle_access.tif',
 '../../data\\access\\upgrade//31m\\14_Kohistan_upgrade_dry_education_high_access.tif',
 '../../data\\access\\upgrade//31m\\17_Torghar_upgrade_dry_health_private_access.tif',
 '../../data\\access\\upgrade//31m\\19_Hangu_upgrade_msn_District_HQs_access.tif',
 '../../data\\access\\upgrade//31m\\20_Hangu_upgrade_msn_education_boys_middle_access.tif',
 '../../data\\access\\upgrade//31m\\22_Hangu_upgrade_msn_education_high_access.tif',
 '../../data\\access\\upgrade//31m\\24_Karak_upgrade_msn_health_private_access.tif',
 '../../data\\access\\upgrade//31m\\26_Karak_upgrade_winter_District_HQs_access.tif',
 '../../data\\access\\upgrade//31m\\28_Karak_upgrade_winter_education_boys_middle_access.tif',
 '../../data\\access\\upgrade//31m\\30_Karak_upgrade_winter_education_high_access.tif',
 '../../data\\access\\upgr

In [23]:
status_quo_road_list[::10]

['../../data\\access\\current//master\\Current_dry_District_HQs.tif',
 '../../data\\access\\current//master\\Current_dry_education_girls_middle.tif',
 '../../data\\access\\current//master\\Current_dry_health_secondary.tif',
 '../../data\\access\\current//master\\Current_msn_education_boys_middle.tif',
 '../../data\\access\\current//master\\Current_msn_health_pharmacies.tif',
 '../../data\\access\\current//master\\Current_winter_education_allboys.tif',
 '../../data\\access\\current//master\\Current_winter_education_high.tif',
 '../../data\\access\\current//master\\Current_winter_markets_All.tif']

## Populate dictionaries for processing

Most of the pertinent information for roads is included in their file names: e.g., `dry_hospitals_rd23_210701` indicates access to hospitals in the dry season for Road 23 <br> We want to create dictionaries from this information that we can later populate with access improvement statistics per road/season/service

In [25]:
proposed_tifs = {}
status_quo_tifs = {}

for i in proposed_road_list:
    
    elements = re.search(r'([0-9]+)_(.*?)_upgrade_(.*?)_(.*?)_access',os.path.basename(i))
    sn = int(elements[1])
    district = elements[2]
    season = elements[3]
    service = elements[4]
    full_id = str(sn) + '_' + season + '_' + service
    
    proposed_tifs[full_id] = {'acc_file': i,\
                           'season': season, 'service': service, 'road_id': sn, \
                           'status_quo' : '', 'absolute_improvement': '', 'relative_gain' : '',\
                           'clip_coords' : '', 'clip_gdf' : '',\
                           'SQ_acc_file' : '', 'dif_acc_file' : '', 'status_quo_file' : '', 'pop_dif_file' : ''}  

    
for i in status_quo_road_list:
    
    elements = re.search(r'Current_(.*?)_(.*?).tif',os.path.basename(i))
    season = elements[1]
    service = elements[2]
    road_id = 'current'
    full_id = season + '_' + service

    status_quo_tifs[full_id] = {'acc_file': i,\
                           'season': season, 'service': service,\
                           'road_id': road_id, 'absolute_improvement': ''}

In [26]:
pprint(dict(islice(proposed_tifs.items(),2)))

{'10_dry_District_HQs': {'SQ_acc_file': '',
                         'absolute_improvement': '',
                         'acc_file': '../../data\\access\\upgrade//31m\\10_LowerChitral_upgrade_dry_District_HQs_access.tif',
                         'clip_coords': '',
                         'clip_gdf': '',
                         'dif_acc_file': '',
                         'pop_dif_file': '',
                         'relative_gain': '',
                         'road_id': 10,
                         'season': 'dry',
                         'service': 'District_HQs',
                         'status_quo': '',
                         'status_quo_file': ''},
 '10_dry_Provincial_HQ': {'SQ_acc_file': '',
                          'absolute_improvement': '',
                          'acc_file': '../../data\\access\\upgrade//31m\\10_LowerChitral_upgrade_dry_Provincial_HQ_access.tif',
                          'clip_coords': '',
                          'clip_gdf': '',
                

In [27]:
pprint(dict(islice(status_quo_tifs.items(),2)))

{'dry_District_HQs': {'absolute_improvement': '',
                      'acc_file': '../../data\\access\\current//master\\Current_dry_District_HQs.tif',
                      'road_id': 'current',
                      'season': 'dry',
                      'service': 'District_HQs'},
 'dry_Provincial_HQ': {'absolute_improvement': '',
                       'acc_file': '../../data\\access\\current//master\\Current_dry_Provincial_HQ.tif',
                       'road_id': 'current',
                       'season': 'dry',
                       'service': 'Provincial_HQ'}}


## Difference calculation

This routine compares each proposed road sequence's accessibility surface to the status_quo accessibility surface and outputs a difference surface (in units of hours of travel)

Activate a loop that calculates the difference in travel time to each service/season category between the post- and pre-investment states.

In [ ]:
for SQ_idx, (key_e, value_e )in enumerate(status_quo_tifs.items()):

    idx = 1 
    
    with rasterio.open(value_e['acc_file'], 'r') as SQ_src:
        SQ_raw = SQ_src.read(1)
        SQ_raw = np.where(SQ_raw > 72, 72, SQ_raw) # cap artificially high values, artifacts of modeling process
        SQ_meta = SQ_src.profile
        
    # countdown
    tPrint(f'{SQ_idx+1} of {len(status_quo_tifs)}, {key_e}')
    
    for prop_idx, (key_p, value_p) in enumerate(proposed_tifs.items()):

        sn = value_p['road_id']
        
        # calculate difference when the season and service are same for status_quo and purposed rasters
        if not value_p['clip_coords'] and value_e['season'] == value_p['season'] and value_e['service'] == value_p['service']:

            with rasterio.open(value_p['acc_file']) as proposed_rd:
                proposed_raw = proposed_rd.read(1)
                proposed_raw = np.where(proposed_raw > 72, 72, proposed_raw)
                proposed_meta = proposed_rd.profile
                
                # create a bbox for clipping
                bds = proposed_rd.bounds
                proposed_bbox = box(bds[0],bds[1],bds[2],bds[3])
                
            # clip the SQ_raw in memory to the extent of the proposed road's analysis area, to speed up processing
            # trying to turn this into a function

            SQ_mask, SQ_mask_tform = rast_ops.clip_in_memory(SQ_raw,SQ_meta,[proposed_bbox],reference_img=proposed_raw)
            
            # our friction surface clipping means that values at the edges of the fric surface are HIGHER b/c roads beyond the clip boundary are missing
            # we assume that higher access values (times) for upgraded roads are always incorrect and remove them (so only equal and lower values are preserved)

            proposed_raw = np.where((proposed_raw > SQ_mask), SQ_mask, proposed_raw)
            
            if np.nansum(np.subtract(SQ_mask,proposed_raw)) > 0:

                dif, value_p['clip_coords'], value_p['clip_gdf'] = rast_ops.dif_clip_calc(SQ_mask, proposed_raw, SQ_mask_tform,dest_crs)

                # Clip the difference array by the extent of the actual differences in the rasters
                # use a MemoryFile to avoid lots of IO -- otherwise have to save down, then load back up.
        
                dif_clip_meta = proposed_meta.copy()
                dif_clip_meta.update({
                    "height" : dif.shape[0],
                    "width" : dif.shape[1],
                    "transform" : SQ_mask_tform
                })
            
                # clip the status_quo and dif layers down to the actual extent of differences, to save on file space and I/O
                dif_clip, dif_clip_tform = rast_ops.clip_in_memory(dif,dif_clip_meta,[value_p['clip_coords']],reference_img=None)
                SQ_export, SQ_export_tform = rast_ops.clip_in_memory(SQ_raw,SQ_meta,[value_p['clip_coords']],reference_img=dif_clip)

                # prepare clipped parameters for export
                export_meta = proposed_meta.copy()
                export_meta.update({
                    "blockxsize": '16',
                    "blockxsize": '16',
                    "height": dif_clip.shape[0],
                    "width": dif_clip.shape[1],
                    "transform": dif_clip_tform
                })

                # Prepare file names and save them to the dict for use in the next step, multiplying by a population raster
                
                status_quo_acc_path = os.path.abspath(os.path.join(data_dir,dif_dir) + '/' + key_p + '_status_quo_access.tif')
                dif_path = os.path.abspath(os.path.join(data_dir,dif_dir) + '/' + key_p + '_post_upgrade_access.tif')
                
                value_p['SQ_acc_file'] = status_quo_acc_path
                value_p['dif_acc_file'] = dif_path

                # write out clipped status_quo and proposed -- using proposed_meta for both
    
                with rasterio.open(status_quo_acc_path, "w", **export_meta) as out_status_quo_acc:
                    out_status_quo_acc.write(SQ_export,indexes=1)  
                
                with rasterio.open(dif_path, "w", **export_meta) as out_dif:
                    out_dif.write(dif_clip,indexes=1)   # ,indexes=1

                # count up
                idx += 1    

            else:
                None

## Calculate access improvement (person-hours) for each dif layer

In [91]:
pop_fil_path = os.path.join(data_dir,r'rast_inputs//kp_general_v15_32642_aligned.tif')

In [92]:
for SQ_idx, (key_e, value_e )in enumerate(status_quo_tifs.items()):

    idx = 1 

    # countdown
    tPrint(f'{SQ_idx+1} of {len(status_quo_tifs)}, {key_e}')
    
    # opening pop file one time to avoid repeat IO
    
    with rasterio.open(pop_fil_path) as pop:

        for prop_idx, (key_p, value_p) in enumerate(proposed_tifs.items()):

            sn = value_p['road_id']
        
            # calculate difference when the season and service are same for status_quo and purposed rasters

            if not value_p['absolute_improvement'] and value_e['season'] == value_p['season'] and value_e['service'] == value_p['service'] and value_p['dif_acc_file'] != '':

            # Read in clipped status_quo and proposed layers

                with rasterio.open(value_p['SQ_acc_file']) as SQ_acc_surf:
                    SQ_acc = SQ_acc_surf.read()
                    SQ_acc_meta = SQ_acc_surf.profile

                with rasterio.open(value_p['dif_acc_file']) as proposed_rd:
                    proposed_dif = proposed_rd.read()
                    proposed_meta = proposed_rd.profile

                # Read in population layer, clip by dif layer
                    pop_clip, pop_clip_transform = mask(pop,[value_p['clip_coords']],crop=True)
                    
                    mask_meta = pop.profile
                    mask_meta.update({"driver": "GTiff",
                      "height": proposed_dif.shape[1],
                      "width": proposed_dif.shape[2],
                      "transform": pop_clip_transform})
                    
                    # sometimes the mask is padded with a null value, despite effort not to pad. Manually cut it down to the correct size
                    pop_clip = rast_ops.reference_slicer(pop_clip,proposed_dif)

                # Multiply the results by the underlying population, record the sums as the current and post-upgrade change in person_hours
                # VERY IMPORTANT: This assumes the pop_clip is of the same resolution and shape as the proposed_dif.
                # You must run the "Align_Rasts" notebook on the population data beforehand to ensure the resolution is aligned. The reference slicer will correct any padding issues
                
                if SQ_acc.shape != pop_clip.shape:
                    SQ_acc = rast_ops.reference_slicer(SQ_acc,pop_clip)
                    
                if proposed_dif.shape != pop_clip.shape:
                    proposed_dif = rast_ops.reference_slicer(proposed_dif,pop_clip)
                    
                ## current
                SQ_acc = np.where(SQ_acc > 10000, 10, SQ_acc) # correct absurd values, artifacts of modeling process
                current_person_hours = np.multiply(SQ_acc,pop_clip)
                current_person_hours = current_person_hours.astype('float64')
                current_person_hours[current_person_hours < 0] = 0 # change tiny negatives or null -99999 to 0 to avoid problems with calculations
                value_p['status_quo'] = np.nansum(current_person_hours)
                
                ## change
                proposed_dif = np.where(proposed_dif > 100, 10, proposed_dif) # correct absurd values, artifacts of modeling process
                change_person_hours = np.multiply(proposed_dif,pop_clip)
                change_person_hours = change_person_hours.astype('float64')
                change_person_hours[change_person_hours < 0] = 0 # change tiny negatives or null -99999 to 0 to avoid problems with calculations
                value_p['absolute_improvement'] = np.nansum(change_person_hours)

                # Export final products

                status_quo_ph_path = os.path.abspath(os.path.join(data_dir,ph_dir) + '/' + key_p + '_personhour_statusquo.tif')
                change_ph_path = os.path.abspath(os.path.join(data_dir,ph_dir) + '/' + key_p + '_personhour_dif.tif')

                value_p['status_quo_file'] = status_quo_ph_path # populating the paths makes it easier to trace back results to files for troubleshooting purposes
                value_p['pop_dif_file'] = change_ph_path

                with rasterio.open(status_quo_ph_path, "w", **mask_meta) as out_ph_SQ:
                    out_ph_SQ.write(current_person_hours)
                    
                with rasterio.open(change_ph_path, "w", **mask_meta) as out_ph_dif:
                    out_ph_dif.write(change_person_hours)

                # count up
                idx += 1
                
            else:
                None

07:49:19	1 of 24, dry_District_HQs
07:49:19	2 of 24, dry_Provincial_HQ
07:49:19	3 of 24, dry_education_allboys
07:49:21	4 of 24, dry_education_allgirls
07:49:22	5 of 24, dry_education_boys
07:49:23	6 of 24, dry_education_boys_high
07:49:24	7 of 24, dry_education_boys_middle
07:49:26	8 of 24, dry_education_boys_primary
07:49:27	9 of 24, dry_education_girls
07:49:28	10 of 24, dry_education_girls_high
07:49:29	11 of 24, dry_education_girls_middle
07:49:31	12 of 24, dry_education_girls_primary
07:49:32	13 of 24, dry_education_high
07:49:32	14 of 24, dry_education_middle
07:49:33	15 of 24, dry_education_primary
07:49:35	16 of 24, dry_health_family
07:49:35	17 of 24, dry_health_pharmacies
07:49:36	18 of 24, dry_health_primary
07:49:37	19 of 24, dry_health_private
07:49:38	20 of 24, dry_health_public
07:49:40	21 of 24, dry_health_secondary
07:49:41	22 of 24, dry_health_tertiary
07:49:41	23 of 24, dry_markets_All
07:49:41	24 of 24, dry_markets_Central


Export the final dict as a pickle, just in case!

In [93]:
today = datetime.today().strftime('%y%m%d')

In [94]:
import pickle

pickle.dump(proposed_tifs,open(os.path.join(data_dir,tab_dir,f'processed//proposed_tifs_{today}.pickle'),"wb"))

## Prepare and export final, tabular figures

Generate sums from the dictionary

In [208]:
# prop_tifs_final = proposed_tifs.copy() # if skipping the pickling step
prop_tifs_final = pickle.load(open(os.path.join(data_dir,tab_dir,f'processed//proposed_tifs_{today}.pickle'),'rb'))

C:\Users\wb467985\Anaconda3\envs\geo\lib\site-packages\pyproj\crs\crs.py:292: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


In [209]:
# rework dict to a dataframe and export a backup copy
df = pd.DataFrame(prop_tifs_final)
df = df.transpose()
df.to_csv(os.path.join(data_dir,tab_dir,f'raw//raw_roads_data_{today}.csv'))

In [210]:
df.dtypes

acc_file           object
season             object
service            object
road_id            object
status_quo         object
person_hours       object
relative_gain      object
clip_coords        object
clip_gdf           object
exist_acc_file     object
dif_acc_file       object
status_quo_file    object
pop_dif_file       object
dtype: object

In [219]:
# clean up and calculate final improvement / status quo stats
df['absolute_improvement'] = np.where(df.absolute_improvement == '', 0, df.absolute_improvement).astype(np.float32)
df['status_quo'] = np.where(df.status_quo == '', 0, df.status_quo).astype(np.float32)
df['relative_access_improvement'] = df['absolute_improvement'] / df['status_quo']

Create wide datasets and join them to the proposed roads data

In [221]:
# Prepare wide datasets
# status quo

sq = df[['season','service','road_id','status_quo']].reset_index()
sq.rename({'status_quo':'access_status_quo','index':'full_string'},axis=1,inplace=True)
sq.access_status_quo = sq.access_status_quo.replace({'':0}).astype(np.float32)

sq_wide = sq.pivot_table(index = 'road_id',\
               columns=['season','service'],\
               values = 'access_status_quo')

# relative access improvement

rai = df[['season','service','road_id','absolute_improvement']].reset_index()
rai.rename({'relative_improvement':'relative_access_improvement','index':'full_string'},axis=1,inplace=True)
rai.access_improvement = rai.access_improvement.replace({'':0}).astype(np.float32)

rai_wide = rai.pivot_table(index = 'road_id',\
               columns=['season','service'],\
               values = 'relative_access_improvement')

# absolute access improvement

aai = df[['season','service','road_id','absolute_improvement']].reset_index()
aai.rename({'absolute_improvement':'absolute_access_improvement','index':'full_string'},axis=1,inplace=True)
aai.access_improvement = aai.access_improvement.replace({'':0}).astype(np.float32)

aai_wide = aai.pivot_table(index = 'road_id',\
               columns=['season','service'],\
               values = 'absolute_access_improvement')

In [222]:
ai.dtypes

full_string            object
season                 object
service                object
road_id                object
access_improvement    float32
dtype: object

In [223]:
sq_wide.head()

season            dry                                                     \
service  District_HQs Provincial_HQ education_allboys education_allgirls   
road_id                                                                    
1        1.035284e+06     5494312.5        168.504105         775.723999   
2        1.689520e+06     6939172.5      10629.712891       12685.552734   
3        1.550664e+06    12034742.0       1891.750488        3259.136963   
4        2.535394e+06     4773613.0       3324.003906        7857.154785   
5        5.007174e+05     4201633.0        566.588928         622.045471   

season                                                            \
service education_boys education_boys_high education_boys_middle   
road_id                                                            
1           168.504105         1807.461182          43734.132812   
2         10629.712891        19078.341797         101762.195312   
3          1891.750488         6823.868652           5461.372559   
4          3324.003906        28965.160156          16418.210938   
5           566.588928        29570.373047           2634.598389   

season                                                               ...  \
service education_boys_primary education_girls education_girls_high  ...   
road_id                                                              ...   
1                  3678.591797      775.723999         13015.161133  ...   
2                 11217.901367    12685.552734         54482.664062  ...   
3                  2050.009277     3259.136963        113598.429688  ...   
4                  3617.860107     7857.154785         16715.947266  ...   
5                   570.408691      622.045471          1749.673462  ...   

season                                                                     \
service education_primary  health_family health_pharmacies health_primary   
road_id                                                                     
1              157.388062   14916.991211       8443.415039    6787.449219   
2             8610.329102   56340.351562      54930.535156   57795.226562   
3             2006.568970   92196.734375      27986.630859    8784.625977   
4             3569.731445   20431.742188      15053.243164   16317.101562   
5              548.164551  321780.156250       2031.076172    1783.548462   

season                                                                 \
service health_private health_public health_secondary health_tertiary   
road_id                                                                 
1         16224.124023  35514.628906     15710.167969      14888693.0   
2         68734.031250  37670.597656     66571.718750      16611544.0   
3        117987.484375   5512.084961    880414.750000       9974145.0   
4         19534.412109  27829.070312     18302.658203       9965408.0   
5          1818.155762   1546.508057      1625.170532       9969713.0   

season                                 
service   markets_All markets_Central  
road_id                                
1        1.627323e+04      11211585.0  
2        2.635565e+06      28946200.0  
3        1.175497e+05      11211585.0  
4        1.826460e+04      11211585.0  
5        1.636591e+03      25922172.0  

[5 rows x 24 columns]

In [224]:
aai_wide.head()

season             dry                                                      \
service   District_HQs  Provincial_HQ education_allboys education_allgirls   
road_id                                                                      
1         31276.365234  145493.156250          0.574245           5.970305   
2        125080.671875  153855.703125        540.868286        1044.523193   
3         40349.496094  162369.125000        186.048492         333.995056   
4         54962.996094  145702.906250        238.853745        1665.343750   
5         26672.503906  144784.968750         10.569116           7.493680   

season                                                            \
service education_boys education_boys_high education_boys_middle   
road_id                                                            
1             0.574245           10.605741             46.245304   
2           540.868286         2996.606689           9186.515625   
3           186.048492         1646.107178            583.048279   
4           238.853745         2308.705322           1745.625122   
5            10.569116         2595.798584             36.170605   

season                                                               ...  \
service education_boys_primary education_girls education_girls_high  ...   
road_id                                                              ...   
1                   334.003143        5.970305          1204.316772  ...   
2                   617.977539     1044.523193          9448.099609  ...   
3                   224.035660      333.995056         12786.723633  ...   
4                   254.401276     1665.343750          1054.746094  ...   
5                    10.835896        7.493680            16.750071  ...   

season                                                                    \
service education_primary health_family health_pharmacies health_primary   
road_id                                                                    
1                0.719626   1188.181274       1177.813599     191.406784   
2              548.838501   9039.630859      13391.520508    7940.701660   
3              224.035660  17015.691406       2493.569580    2361.935303   
4              167.643402   1161.656372        957.084167    1030.416748   
5               10.782003   8543.295898        203.559586      41.686119   

season                                                                 \
service health_private health_public health_secondary health_tertiary   
road_id                                                                 
1          1116.981934   1572.746948      1194.135742     598539.0000   
2          8828.002930   6384.974609      8828.002930     630214.5625   
3         17515.343750    806.663330     54271.730469     529921.1875   
4          1161.656372   2260.821045      1161.656372     513254.9375   
5            17.080681     16.288937        16.386080     512337.0000   

season                                 
service   markets_All markets_Central  
road_id                                
1         1208.493042    283109.25000  
2        25351.650391    595592.81250  
3        17742.789062    299985.21875  
4         1161.656372    283319.00000  
5           17.080683    506645.40625  

[5 rows x 24 columns]

Join the dataframe to the proposed roads DF

In [225]:
# Load proposed roads data -- make sure to insert correct date for final proposed roads file
prop_rds_gdf_pth = f'Proposed_final//Proposed_roads_processed_{data_date}.gpkg'

In [226]:
proposed_rds = pd.DataFrame(gpd.read_file(os.path.join(data_dir,rds_dir,prop_rds_gdf_pth),driver="GPKG"))

In [227]:
# minor cleanup in preparation for joining
proposed_rds.drop('geometry',axis=1,inplace=True)
proposed_rds['Road_ID'] = proposed_rds.SN.astype(int)
proposed_rds = proposed_rds.set_index('Road_ID')

In [228]:
# filter out unnecessary columns, if relevant
proposed_rds = proposed_rds.iloc[:,6:32]

In [229]:
proposed_rds.index

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 17, 18,
            19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36,
            37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 55,
            56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 70, 71, 72, 73, 74,
            75, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87],
           dtype='int64', name='Road_ID')

In [230]:
aai_wide.index

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 17, 18,
            19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36,
            37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 55,
            56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 70, 71, 72, 73, 74,
            75, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87],
           dtype='int64', name='road_id')

In [231]:
# Join proposed roads dataset to access statistics
prop_rds_sq_final = pd.merge(proposed_rds,sq_wide,how='left',left_index=True,right_index=True)
prop_rds_rel_acc_imp_final = pd.merge(proposed_rds,rai_wide,how='left',left_index=True,right_index=True)
prop_rds_abs_acc_imp_final = pd.merge(proposed_rds,aai_wide,how='left',left_index=True,right_index=True)

C:\Users\wb467985\AppData\Local\Temp/ipykernel_51668/2217510834.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  prop_rds_sq_final = pd.merge(proposed_rds,sq_wide,how='left',left_index=True,right_index=True)
C:\Users\wb467985\AppData\Local\Temp/ipykernel_51668/2217510834.py:3: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  prop_rds_rel_acc_imp_final = pd.merge(proposed_rds,rai_wide,how='left',left_index=True,right_index=True)
C:\Users\wb467985\AppData\Local\Temp/ipykernel_51668/2217510834.py:4: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  prop_rds_abs_acc_imp_final = pd.merge(proposed_rds,ai_wide,how='left',left_index=True,right_index=True)


Check out data

In [233]:
prop_rds_abs_acc_imp_final.tail(2)

,District,Current_Road_Cond,Current_Surface,Current_Road_Class,Upgrade_Road_Cond,Upgrade_Surface,Upgrade_Road_Class,Terrain,Cost_PKR,current_base_speed,...,"(dry, education_primary)","(dry, health_family)","(dry, health_pharmacies)","(dry, health_primary)","(dry, health_private)","(dry, health_public)","(dry, health_secondary)","(dry, health_tertiary)","(dry, markets_All)","(dry, markets_Central)"
Road_ID,,,,,,,,,,,,,,,,,,,,,
86,North Waziristan,Poor,Gravel,District Road,Very Good,Asphaltic,District Road,Plains,4.480068e+08,50.0,...,1.645315e+02,120952.578125,12565.527344,13666.895508,22189.933594,12380.724609,20563.966797,150361.78125,22603.787109,175664.56250
87,North Waziristan,Poor,Gravel,District Road,Very Good,Asphaltic,District Road,Plains,4.766536e+08,50.0,...,1.385636e+06,115366.250000,10776.630859,11762.242188,20249.955078,10027.534180,8887.242188,148315.96875,21894.519531,173078.96875


In [234]:
prop_rds_sq_final.tail(2)

,District,Current_Road_Cond,Current_Surface,Current_Road_Class,Upgrade_Road_Cond,Upgrade_Surface,Upgrade_Road_Class,Terrain,Cost_PKR,current_base_speed,...,"(dry, education_primary)","(dry, health_family)","(dry, health_pharmacies)","(dry, health_primary)","(dry, health_private)","(dry, health_public)","(dry, health_secondary)","(dry, health_tertiary)","(dry, markets_All)","(dry, markets_Central)"
Road_ID,,,,,,,,,,,,,,,,,,,,,
86,North Waziristan,Poor,Gravel,District Road,Very Good,Asphaltic,District Road,Plains,4.480068e+08,50.0,...,4442.694336,3119017.75,1103780.125,1280328.500,1608397.500,524412.25,1.189860e+06,11487173.0,1940999.25,10433678.0
87,North Waziristan,Poor,Gravel,District Road,Very Good,Asphaltic,District Road,Plains,4.766536e+08,50.0,...,58550.738281,3118443.50,1219092.375,1324451.625,1635802.125,647083.50,8.722957e+05,11487173.0,2154267.00,10433678.0


In [232]:
len(prop_rds_abs_acc_imp_final)

79

**Export the final DataFrames to CSVs**

In [2]:
# file names
out_sq_tab_name = f'processed//roads_acc_status_quo_{today}.csv'
out_rel_acc_imp_tab_name = f'processed//roads_relative_acc_improvements_{today}.csv'
out_abs_acc_imp_tab_name = f'processed//roads_absolute_acc_improvements_{today}.csv'

In [240]:
# export
prop_rds_sq_final.to_csv(os.path.join(data_dir,tab_dir,out_sq_tab_name))
prop_rds_rel_acc_imp_final.to_csv(os.path.join(data_dir,tab_dir,out_rel_acc_imp_tab_name))
prop_rds_abs_acc_imp_final.to_csv(os.path.join(data_dir,tab_dir,out_abs_acc_imp_tab_name))

You are done with spatial analysis, congrats! Move on to processing the tabular data however you need for your use case.